In [7]:
#Functions
#Spearman correlation function
#qqplot function
#KSTest function

In [8]:
import xarray as xr
import numpy as np
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
from scipy.stats import kstest
import pandas as pd
import numpy as np

Calculating Spearman Correlation Between two times series (Eg TabsD and RhiresD)

In [9]:
def plot_spearman(file1, var1, file2,var2,title=None, cmap="coolwarm"):
    """This is a function written to plot spearman correlation between two input files wehre:
    file1:input file1 , eg TabsD for average daily temperature data
    file2: input file2, eg RhyresD for  daily precipitation sum data
    var1: variable extracted from file1
    var2: variable extracted from file2
    title: title of the plot
    cmap: colormap for the plot
    """
    #Loading datasets
    ds1= xr.open_dataset(file1)
    ds2= xr.open_dataset(file2)

    #Variables extraction from two files
    var1 = ds1[var1]
    var2 = ds2[var2]

    #Taking care of grid issues in case var1 and var2 dont lie on the same grid
    #Regridding var2 to the grid of var1 (might not be needed)
    if var1.shape != var2.shape:
        var2=var2.interp(lat=var1.lat, lon =var1.lon, method="nearest")

        #Calculating SpearmanR

        def spearman (x,y):

            """This function returns spearman correlation between two variables if they are not NaNs and returns a NaN if either of them is a NaN"""
            mask = ~np.isnan(x) & ~np.isnan(y)
            if mask.sum() > 1:
                return spearmanr(x[mask], y[mask])[0]
            else:
                return np.nan
            
    #Now the spearman function is applied to the two variables
        spearman_corr= xr.apply_ufunc(spearman, var1, var2,
                                        input_core_dims=[['time'], ['time']],
                                        vectorize=True,
                                        dask='allowed',
                                        output_dtypes=[np.float64])
    #Plotting results obtained from the gridwise calculation
        plt.figure(figsize=(10, 6))
        spearman_corr.plot(cmap=cmap, add_colorbar=True)
        plt.title(title if title else f"Spearman Correlation between {var1.name} and {var2.name}")
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.show()

Calculating and Displaying the QQ plot(spatial averaged in time) between two variables (eg TabsD and RhiresD)

In [10]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from statsmodels.graphics.gofplots import qqplot_2samples


def qqplot(file1, var1, file2,var2,title=None):
    """This function intends to generate the porbabilistic QQplot between two distributions, by first spatially averaging the data over the region of interest
    - file1, file2: paths to NetCDF files
    - var1, var2: eg TabsD and RhiresD for daily temp and precip for example
    - title: title for the plot, can be customised or left as None
    """

    #Loading the two datasets
    ds1 = xr.open_dataset(file1)
    ds2 = xr.open_dataset(file2)

    #Extracting the variables from the two datasets
    var1 = ds1[var1]   
    var2 = ds2[var2]

    #Spatial averaging, over lat and lon, skipping NaNs
    var1_ts=var1.mean(dim=["lat","lon"], skipna=True).values
    var2_ts=var2.mean(dim=["lat","lon"], skipna=True).values

    #Drop NaNs if any
    mask = ~np.isnan(var1_ts) & ~np.isnan(var2_ts)
    var1_ts1 = var1_ts[mask]
    var2_ts2 = var2_ts[mask]

    #Plotting QQplot using the statsmodels library functionalities
    plt.figure(figsize=(10,6))
    qqplot_2samples(var1_ts,var2_ts, line="45") #Includes the 45 degrees line
    plt.xlabel(f"Quantiles of spatially averaged {var1}" )
    plt.ylabel(f"Quantiles of spatially averaged {var2}" )
    plt.title(title or f"QQ Plot : {var2} vs {var1}")
    plt.grid(False)
    plt.tight_layout()
    plt.show()





Calculating and displaying the gridded QQ slope (slope of the regression line) for enabling gridded representation of QQPlot metrics

In [12]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from statsmodels.graphics.gofplots import qqplot_2samples
from scipy.stats import linregress

def qq_slope(file1, var1, file2, var2, title=None, cmap="coolwarm", quantiles=np.linspace(0.05, 0.95, 19),center_colorbar=False):
    """Computes and plots the gridded spatial map of QQ slopes based on probabilistic QQ plot
    Inputs: 
    file1,file2: NetCDF file paths
    var1,var2= variable names from the file1 and file2 resp. eg. TabsD and RhiresD
    title: optional plot title
    cmap: colorbar
    quantiles: probabilities 
    center_colorbar : False by default, can be changed to True if one needs colorbar for the gridded map of slopes centered around 0"""


    #Loading the datasets
    ds1= xr.open_dataset(file1)
    ds2= xr.open_dataset(file2)
    v1= ds1[var1]
    v2= ds2[var2]

    #Regridding might be required if the datasets are not of the same shapes
    
    if v1.shape!= v2.shape:
        v2= v2.interp_like (v1, method = "nearest")

    def qq_slope_from_probabilistic_quantiles(x, y):
        """Calculate QQ slope using probabilistic quantiles."""
        mask = ~np.isnan(x) & ~np.isnan(y)
        x = x[mask]
        y = y[mask]
        if len(x) < 10 or len(y) < 10:
            return np.nan  # not enough data

        try:
            
            qx = np.percentile(x, quantiles * 100) 
            qy = np.percentile(y, quantiles * 100)
            slope, _, _, _, _ = linregress(qx, qy)
            return slope
        except Exception:
            return np.nan

    # Applying function grid-wise to calculate the QQ slope
    slope_map = xr.apply_ufunc(
        qq_slope_from_probabilistic_quantiles, v1, v2,
        input_core_dims=[["time"], ["time"]],
        vectorize=True,
        dask="allowed",
        output_dtypes=[np.float64]
    )

    # Plot
    plt.figure(figsize=(10, 6))
    
    if center_colorbar:
        vmin = -np.nanmax(np.abs(slope_map.values))
        vmax = np.nanmax(np.abs(slope_map.values))
    else:
        vmin = slope_map.min().values
        vmax = slope_map.max().values
    
    slope_map.plot(cmap=cmap, add_colorbar=True, vmin=vmin, vmax=vmax)

    plt.title(title or f"Grid-wise QQ Slope: {var1} vs {var2}")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.grid(False)
    plt.tight_layout()
    plt.show()
    return slope_map


The Kalmogorov Smirnov Test statistic compares two distributions (RhiresD and TabsD for example) to determine how similar or different they are based on the largest distance between the Empirical CDFs (the step function) of the two distributions

In this case, the Null hypothesis: two datasets come from the same underlying distributions

Alternative Hypothesis : both datasets come from different underlying distributions

Implementation in Python : with the scipy.stats.ks_2samp() , which calculates KS statistic and also the p value for statistical significance